In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from patsy import dmatrices
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error 

In [4]:
df = pd.read_csv('CAS_modified .csv')
pd.set_option('display.max_columns',None)
df.head()

,Unnamed: 0,X,Y,bicycle,bridge,bus,carStationWagon,cliffBank,crashDirectionDescription,crashFinancialYear,crashLocation1,crashLocation2,crashSeverity,crashSHDescription,crashYear,debris,directionRoleDescription,ditch,fatalCount,fence,flatHill,guardRail,houseOrBuilding,kerb,light,minorInjuryCount,moped,motorcycle,NumberOfLanes,objectThrownOrDropped,otherObject,otherVehicleType,overBank,parkedVehicle,phoneBoxEtc,postOrPole,region,roadCharacter,roadLane,roadSurface,roadworks,schoolBus,seriousInjuryCount,slipOrFlood,speedLimit,strayAnimal,streetLight,suv,taxi,tlaName,trafficControl,trafficIsland,trafficSign,train,tree,truck,unknownVehicleType,urban,vanOrUtility,vehicle,waterRiver,weatherA
0,0,1772561.0,5896382.0,0.0,0.0,0.0,1.0,0.0,None,2007/2008,EAST ST,ELLIOT ST,Non-Injury Crash,No,2007,0.0,North,0.0,0.0,0.0,Flat,0.0,0.0,0.0,Dark,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Auckland Region,Nil,2-way,Sealed,0.0,0.0,0.0,0.0,50.0,0.0,On,1.0,0.0,Auckland,Traffic Signals,0.0,0.0,0.0,0.0,0.0,0.0,Urban,0.0,0.0,0.0,Fine
1,1,1836757.0,5859311.0,0.0,0.0,0.0,2.0,0.0,None,2002/2003,SH 2,SH 26,Non-Injury Crash,Yes,2003,0.0,North,0.0,0.0,0.0,Flat,0.0,0.0,0.0,Dark,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Waikato Region,Nil,2-way,Sealed,0.0,0.0,0.0,0.0,50.0,0.0,On,0.0,0.0,Hauraki District,Give way,0.0,0.0,0.0,0.0,0.0,0.0,Urban,0.0,0.0,0.0,Fine
2,2,1762088.0,5912507.0,0.0,0.0,0.0,1.0,0.0,West,2009/2010,CHURCH ST,HUGO JOHNSTON DRIVE,Non-Injury Crash,No,2010,0.0,East,0.0,0.0,0.0,Flat,0.0,0.0,0.0,Bright sun,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Auckland Region,Nil,2-way,Sealed,0.0,0.0,0.0,0.0,50.0,0.0,Off,0.0,0.0,Auckland,Traffic Signals,0.0,0.0,0.0,0.0,0.0,0.0,Urban,1.0,0.0,0.0,Fine
3,3,1753522.0,5911939.0,0.0,0.0,0.0,2.0,0.0,None,2017/2018,THE AVENUE,ACCESSWAY,Minor Crash,No,2017,0.0,North,0.0,0.0,0.0,Hill Road,0.0,0.0,0.0,Overcast,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Auckland Region,Nil,2-way,Sealed,0.0,0.0,0.0,0.0,50.0,0.0,None,0.0,0.0,Auckland,Nil,0.0,0.0,0.0,0.0,0.0,0.0,Urban,0.0,0.0,0.0,Light rain
4,4,1761364.0,5914259.0,0.0,0.0,0.0,2.0,0.0,None,2017/2018,PENROSE INT,GT SOUTH LINK,Non-Injury Crash,No,2018,0.0,North,0.0,0.0,0.0,Flat,0.0,0.0,0.0,Overcast,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Auckland Region,Motorway ramp,1-way,Sealed,0.0,0.0,0.0,0.0,50.0,0.0,None,0.0,0.0,Auckland,Give way,0.0,0.0,0.0,0.0,0.0,0.0,Urban,0.0,0.0,0.0,Fine


In [ ]:
#print(train_data.shape)
#print(test_data.shape)

In [22]:
df['seriousInjuryCount'] = df.apply(
    lambda row: 5 if row['seriousInjuryCount'] >=4 else row['seriousInjuryCount'] ,
    axis=1
)

In [23]:
np.random.seed(99)
mask = np.random.rand(len(df)) <= 0.8
train_data = df[mask]
test_data = df[~mask]

In [24]:
df['seriousInjuryCount'].value_counts()

0.0    730095
1.0     40297
2.0      4349
3.0       865
5.0       385
Name: seriousInjuryCount, dtype: int64

In [25]:
formula = "seriousInjuryCount ~ bicycle+bridge+bus+carStationWagon+cliffBank+debris+ditch+fence+flatHill+guardRail+houseOrBuilding+kerb+light+moped+motorcycle+NumberOfLanes+objectThrownOrDropped+otherObject+otherVehicleType+overBank+parkedVehicle+postOrPole+roadLane+roadSurface+roadworks+schoolBus+slipOrFlood+speedLimit+strayAnimal+streetLight+suv+taxi+trafficIsland+trafficSign+train+tree+truck+unknownVehicleType+urban+vanOrUtility+vehicle+waterRiver+weatherA"
y, X = dmatrices(formula, df, return_type='dataframe')

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report, f1_score

In [27]:
def plot_confusion_matrix(y_true: np.ndarray, y_hat: np.ndarray, figsize = (20, 15)):
    # Getting labels
    labels = sorted(list(set(y_true)))
    
    # Creating confusion matrix
    df = pd.DataFrame(confusion_matrix(y_true, y_hat), index = labels, columns = labels)
    
    # Accuracy and F1-score
    acc = accuracy_score(y_true, y_hat)
    f1s = f1_score(y_true, y_hat, average = 'weighted')
    
    # Ploting Confusin matrix
    fig, ax = plt.subplots(figsize = figsize)
    sns.heatmap(df, annot = True, square = True, fmt = 'd', linewidths = 1,cbar = False, cmap = 'Paired', ax = ax)
    ax.set(title = f'Accuracy: {acc:.2f}, F1 score: {f1s:.2f}', xlabel = 'Predicted', ylabel = 'Actual')
    fig.suptitle('Confusion Matrix')
    plt.tight_layout()

In [32]:
X_train, X_test, y_train, y_test = train_test_split(x,Y,test_size = .3, random_state = 42)

print('Train shape:', X_train.shape)
print('Test shape:', X_test.shape)

Train shape: (1788916, 58)
Test shape: (766679, 58)


In [ ]:
X_train.head()

In [ ]:
y_train.seriousInjuryCount.value_counts()

In [ ]:
import matplotlib.pyplot as plt
plt.hist(y_train.seriousInjuryCount)
plt.show()

In [33]:
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

#logg = LogisticRegression()
#logg.fit(X_train,y_train)


In [34]:
%%time
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
# define the model evaluation procedure
#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
#n_scores = cross_val_score(model, x, Y, scoring='accuracy', cv=cv, n_jobs=-1)
# report the model performance
#print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Wall time: 0 ns


In [35]:
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [38]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(model.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.38


array([5., 5., 5., ..., 0., 5., 5.])

In [ ]:
y_pred1 = model.predict(X_train)
y_pred1.sum()

## Cross Validation

In [ ]:
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
kfold = model_selection.KFold(n_splits=10)
modelCV = LogisticRegression()
scoring = 'accuracy'
results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

In [ ]:
clf.predict_proba(X_train)

In [ ]:
pip install imbalanced-learn

In [ ]:
pip install imblearn


In [21]:
y.seriousInjuryCount.value_counts()

0.0     730095
1.0      40297
2.0       4349
3.0        865
4.0        259
5.0         81
6.0         26
7.0          8
8.0          5
10.0         3
12.0         1
9.0          1
14.0         1
Name: seriousInjuryCount, dtype: int64

In [29]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from collections import Counter

In [30]:
smote=SMOTE('not majority', k_neighbors=3)
x,Y=smote.fit_resample(X_train,y_train)

C:\Users\msc2\anaconda3\lib\site-packages\imblearn\utils\_validation.py:586: FutureWarning: Pass sampling_strategy=not majority as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


In [31]:
Y.value_counts()

seriousInjuryCount
0.0                   511119
1.0                   511119
2.0                   511119
3.0                   511119
5.0                   511119
dtype: int64

In [ ]:
#flatHill[T.Hill Road],light[T.Dark],light[T.Overcast],region[T.Bay of Plenty Region],region[T.Canterbury Region],region[T.Hawke's Bay Region],region[T.Canterbury Region],region[T.Northland Region],roadCharacter[T.Motorway ramp],

### Poisson GLM Regression

In [ ]:
formula = "seriousInjuryCount ~ bicycle+bridge+bus+carStationWagon+cliffBank+crashYear+debris+ditch+fence+flatHill+guardRail+houseOrBuilding+kerb+light+moped+motorcycle+NumberOfLanes+objectThrownOrDropped+otherObject+otherVehicleType+overBank+parkedVehicle+postOrPole+roadLane+roadworks+schoolBus+slipOrFlood+speedLimit+strayAnimal+streetLight+suv+taxi+trafficControl+trafficIsland+trafficSign+train+tree+truck+unknownVehicleType+urban+vanOrUtility+vehicle+waterRiver+weatherA"
y_train, X_train = dmatrices(formula, train_data, return_type='dataframe')
y_test, X_test = dmatrices(formula, test_data, return_type='dataframe')

In [ ]:
model_poisson = sm.GLM(y_train, X_train,family=sm.families.Poisson(link=sm.families.links.log())).fit()

In [ ]:
model_poisson.summary2()

In [ ]:
ypred = model_poisson.predict(X_train)
rmse = np.sqrt(mean_squared_error(y_train,ypred))
rmse

In [ ]:
rmse/(-y_train.min()+y_train.max())

In [ ]:
ypred = model_poisson.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test,ypred))
rmse

In [ ]:
def calculate_residuals(model, features, label):
    predictions = model.predict(features)
    df_results = pd.DataFrame({'Actual': label.squeeze(1), 'Predicted': predictions})
    df_results['Residuals'] = abs(df_results['Actual']) - abs(df_results['Predicted'])
    df_results['Residuals_per'] = (abs(df_results['Actual']) - abs(df_results['Predicted']))/df_results['Actual']
    
    return df_results

In [ ]:
df_results = calculate_residuals(model_poisson,X_train,y_train)
df_results.tail()

In [ ]:
#pip install git+https://github.com/madrury/py-glm.git

In [ ]:
y_train.seriousInjuryCount.value_counts()

In [ ]:
pd.Series(np.round(df_results.Predicted)).value_counts()